In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pickle

import catboost as cb
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

In [24]:
os.listdir("data")

['data_kmeans.csv',
 'data_kmeans_replace.csv',
 'data_n.csv',
 'test',
 'test.txt',
 'UnlabeledWiDS2021.csv']

In [25]:
# data = pd.read_csv("data/data_n.csv")

In [26]:
data = pd.read_csv("data/UnlabeledWiDS2021.csv")

In [27]:
data = data.drop("Unnamed: 0", axis=1)
data["target"] = data["solid_tumor_with_metastasis"]
data = data.drop("solid_tumor_with_metastasis", axis=1)

In [28]:
data_kmeans = data.copy()

In [29]:
categorical = data_kmeans.dtypes[data_kmeans.dtypes == "object"].index
numerical = list(set(data_kmeans.columns) - set(categorical))

print(f"Всего категориальных признаков: {len(categorical)}")
print(f"Всего числовых признаков: {len(numerical)}")


Всего категориальных признаков: 6
Всего числовых признаков: 173


In [30]:
freq_encoder = data_kmeans["gender"].value_counts(normalize=True)
freq_encoder

M    0.540131
F    0.459869
Name: gender, dtype: float64

In [31]:
cat_new = []
for i in data_kmeans[categorical]:
    freq_encoder = data_kmeans[i].value_counts(normalize=True)
    data_kmeans[f"{i}_freq"] = data_kmeans[i].map(freq_encoder)
    cat_new.append(f"{i}_freq")
cat_new

['ethnicity_freq',
 'gender_freq',
 'hospital_admit_source_freq',
 'icu_admit_source_freq',
 'icu_stay_type_freq',
 'icu_type_freq']

In [32]:
# удалю старые признаки типа обьект
data_kmeans = data_kmeans.drop(categorical, axis=1)

In [33]:
# # посмотрим на пропуски и есть ли среди них признаки с уникальными значениями более 9000
# count = 0
# check_list_sign = []

# for i in data_kmeans:
#     isna = data_kmeans[i].isna().sum()
#     if isna > 0:
#         count += 1
#         print(i)
#         check_list_sign.append(i)
#         if data_kmeans[i].nunique() > 9000:
#             print("unique: ", data_kmeans[i].nunique())
#         print(isna)
#         print("*" * 30)

In [34]:
# меняю пропуски на -100
# data_kmeans = data_kmeans.fillna(-100)

In [35]:
# a = list(data_kmeans["bmi"].loc[data_kmeans["bmi_kmeans"] == 100])
# a
# # a.remove(-100)
# # a_m = round(np.median(a), 3)
# # print(a_m)
# # q = data_kmeans["bmi"].loc[data_kmeans["bmi_kmeans"] == 100]
# # q
# # s = np.where(q != -100, q, a_m)
# # data_kmeans["bmi_kmeans"][-100]
# # # q = s
# # q

In [36]:
# kmeans = KMeans(n_clusters=50, random_state=0) # кол-во кластеров равно уникальному значению иследуемой совокупности
# kmeans.fit(data_kmeans) # обучение

# TEST

In [37]:
from collections import Counter

In [38]:
cnt = Counter
a = [0,0,0,0,0,0,1,1,1,1]
c = cnt(a)
print(c[1])


4


In [39]:
data_kmeans_copy = data_kmeans.copy()

In [40]:
def close_funk():
    print("нету списка признаков")
    return

In [41]:
def save_values(df,check_list_sign, count_breack):
        print(f"{count_breack} эпох без изменений")
        print(f"лучший результат: {len(check_list_sign)}")
        df.to_csv("data/data_kmeans_replace_all_rows.csv", index=False)
        with open("data/test", "wb") as fp:
            pickle.dump(check_list_sign, fp)
        print("Программа завершена, данные сохранены")
        return df,check_list_sign


In [42]:
def recursive(df, max_check_col, counts, list_sign=[], true_list_sign=[], count_breack=25):
    """
    ф-я принимает:
    true_list_sign  -  список признаков где уже заменены -100 на медиану
    
    list_sign: list   - список признаков в которых присутствуют замененные пропуски на -100
    
    df : Pandas DataFrame   -   матрица данных
    
    counts : int  -  изначально равно нулю, номер эпохи
    
    max_check_col : int  -  длина листа с признаками в которых остались значения -100
    
    count_breack : int  -  кол-во эпох для остановки если нет улучшений
    
    """
    dataf, check_list_sign, true_list_sign, max_check_col, counts, *a =  check_nan_replace1(df=df,
                                                                         list_sign=list_sign, 
                                                                         true_list_sign=true_list_sign, 
                                                                         max_check_col=max_check_col,
                                                                         counts=counts)
    
    print("check_list_sign: ",check_list_sign)                                                                   
    if "".join(a) == "quit":
#         dataf = dataf.copy()
        print("quit")
        df = dataf
#         print(dataf.shape, check_list_sign)
        save_values(df,check_list_sign, count_breack)
        print(dataf.shape, "len: ", len(check_list_sign))
#         print(dataf.shape, "len: ", len(check_list_sign))
        return (df)
    
    if counts == count_breack: 
        dataf,check_list_sign = save_values(dataf,check_list_sign, count_breack)
        print(dataf.shape, "len: ", len(check_list_sign))
#         print(dataf.shape, "len: ", len(check_list_sign))
        return (dataf, check_list_sign)

    recursive(df=dataf, list_sign=check_list_sign, true_list_sign=true_list_sign, max_check_col=max_check_col, counts=counts)

In [43]:
def counter_target(num_sign, col, df, counter):
    print("я тут")
    print(df[col].loc[(df[col] == num_sign and df["target"] == 1)])
    
#     print(counter)

In [44]:
def check_nan_replace1(df, list_sign=[], true_list_sign=[], max_check_col=1000, counts=0):
    
    print(df.shape)
    if len(list(set(df.isna().sum().values))) != 1:
        list_sign = []

        for i in df:
            isna = df[i].isna().sum()
            if isna > 0:
                list_sign.append(i)
        df = df.fillna(-100)
    if len(list_sign) == 0:
        print("лист с пропусками не задан")
        return df, list_sign, true_list_sign, max_check_col, counts, "quit"
    new_sign = []
    print("1 list_sign: ",len(list_sign))
    print("2 true_list_sign: ",len(true_list_sign))
#     list_sign = [i for i in list_sign if i not in true_list_sign]
    print("3 list_sign: ",len(list_sign))
    for i in df[list_sign].columns: # цикл по столбцам в которых были заменены пропуски на -100
        nuniq = df[i].nunique() # кол-во уникальных значений в признаке
        uniq = df[i].unique()
#         print("4 uniq:",uniq)
#         print("5 if -100 not in uniq: ",-100 not in uniq)
#         if -100 not in uniq:
            
#             true_list_sign.append(i)
#             continue
        
        if nuniq == 2 or (nuniq - counts) <= 1:
            kmeans = KMeans(n_clusters=nuniq, random_state=0) # кол-во кластеров равно уникальному значению иследуемой совокупности
            kmeans.fit(df) # обучение
        else:
            kmeans = KMeans(n_clusters=nuniq - counts, random_state=0) # кол-во кластеров равно уникальному значению иследуемой совокупности
            kmeans.fit(df) # обучение
        if i.split("_")[-1] != "kmeans":
            df[f"{i}_kmeans"] = kmeans.labels_
            new_sign.append(f"{i}_kmeans") # записываю новые признаки в лист
            
        else:
            df[i] = kmeans.labels_
            new_sign.append(i)
    
    check_list_sign = []
    print("6 new_sign:", len(new_sign))
    for col in df[new_sign].columns: # цикл по колонкам которые получились
#         uniq_sign = df[col].nunique() # кол-во кластеров
        
        uniq_sign = 50 # потом удалить
        for num_sign in range(uniq_sign): # цикл по кластерам признака
            check_list = list(set(df["target"].loc[df[col] == num_sign])) # лист уникальных значений целевой переменной
                                                                          # исследуемой совокупности
#             counter = cnt(list(list(df["target"].loc[df[col] == num_sign])))
#             print("minmax: ", max(counter.values()) - min(counter.values()))
#             print("counter: ", counter)
#             if counter[1] >= 5 and counter[0] >=5:
#                 counter_target(num_sign, col, df, counter)
            col_del_pr = col.split("_kmeans")[0]
            if len(check_list) == 1: # если длина листа равна 1

#                 col_del_pr = col.split("_kmeans")[0] # получаю название без префикса _kmeans
                a = list(set(df[col_del_pr].loc[df[col] == num_sign])) # лист с уникальными значениями  
                                                                       # группы исследуеиой совокупности
                if len(a) == 1 and a[0] == -100:
                    check_list_sign.append(col)
                    continue
                try:                                                                         
                    a.remove(-100)  # удаляю -100
                except:
                    pass
                mean_a = round(np.median(a),3) # получаю медиану совокупности без -100
                
                q = df[col_del_pr].loc[df[col] == num_sign] # список значений группы 
                                                                           # исследуеиой совокупности
#                 s = np.where(q != -100, q, mean_a) # меняю -100 на медиану
#                 print("q: ",q)
                s = np.where(q ==-100, mean_a, q)  # меняю -100 на медиану  
#                 print("s: ",s)
                df = df.copy()
#                 df[col_del_pr].loc[df[col] == num_sign] = df[col_del_pr].loc[df[col] == num_sign].replace(-100, mean_a)

                df[col_del_pr].loc[df[col] == num_sign] = s # меняю списки
            elif len(check_list) == 2: # если длина листа равна 2
#                 col_del_pr = col.split("_kmeans")[0]
                for num_t in range(2):
                    a = list(set(df[col_del_pr].loc[df[col] == num_sign].loc[df['target']==num_t])) # лист с уникальными значениями  
                                                                                            # группы исследуеиой совокупности
                    if len(a) == 1 and a[0] == -100:
                        check_list_sign.append(col)
                        continue
                    try:                                                                         
                        a.remove(-100)  # удаляю -100
                    except:
                        pass
                    mean_a = round(np.median(a),3) # получаю медиану совокупности без -100
                    q = df[col_del_pr].loc[df[col] == num_sign].loc[df['target']==num_t] # список значений группы 
                                                                                         # исследуеиой совокупности
#                     s = np.where(q != -100, q, mean_a) # меняю -100 на медиану 
#                     print("q: ",q)
                    s = np.where(q ==-100, mean_a, q)  # меняю -100 на медиану  
#                     print("s: ",s)
                    df = df.copy()
    #                 df[col_del_pr].loc[df[col] == num_sign] = df[col_del_pr].loc[df[col] == num_sign].replace(-100, mean_a)

                    ind = df[col_del_pr].loc[df[col] == num_sign].loc[df['target']==num_t].index # меняю списки
                    df[col_del_pr].iloc[ind] = s
                
#             else:
#                 check_list_sign.append(col)
    check_list_sign = list(set(check_list_sign))
    len_check_list_sign = len(check_list_sign)
    if len_check_list_sign == 0:
        print("нету списка признаков или все признаки пройдены")
        return df, check_list_sign, true_list_sign, max_check_col, counts, "quit"
    
    elif len_check_list_sign < max_check_col:
        print(f"попытка № {counts} успешная, счетчик обновится")
        print(f"len bad cols: {len(set(check_list_sign))}")
        counts = 0
        max_check_col = len_check_list_sign
        print("check_list_sign: ", check_list_sign)
        return df, check_list_sign, true_list_sign, max_check_col, counts
    else:
        counts += 1
        print(f"попытка № {counts} не привела к успеху")
        return df, check_list_sign, true_list_sign, max_check_col, counts
    
        

In [45]:
d= recursive(data_kmeans_copy, 1000, 0)

(10234, 179)
1 list_sign:  159
2 true_list_sign:  0
3 list_sign:  159
6 new_sign: 159


c:\program files\python37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


попытка № 0 успешная, счетчик обновится
len bad cols: 105
check_list_sign:  ['h1_hemaglobin_max_kmeans', 'h1_hco3_max_kmeans', 'h1_diasbp_noninvasive_max_kmeans', 'd1_mbp_invasive_max_kmeans', 'h1_bilirubin_max_kmeans', 'h1_creatinine_min_kmeans', 'h1_platelets_max_kmeans', 'h1_platelets_min_kmeans', 'd1_arterial_pco2_max_kmeans', 'h1_mbp_noninvasive_max_kmeans', 'h1_potassium_min_kmeans', 'd1_glucose_min_kmeans', 'h1_albumin_max_kmeans', 'd1_pao2fio2ratio_min_kmeans', 'h1_pao2fio2ratio_min_kmeans', 'ph_apache_kmeans', 'pao2_apache_kmeans', 'h1_inr_min_kmeans', 'h1_hematocrit_min_kmeans', 'h1_sodium_min_kmeans', 'bilirubin_apache_kmeans', 'h1_arterial_pco2_max_kmeans', 'd1_hco3_min_kmeans', 'd1_creatinine_min_kmeans', 'd1_wbc_min_kmeans', 'd1_albumin_max_kmeans', 'albumin_apache_kmeans', 'h1_sysbp_invasive_max_kmeans', 'h1_wbc_min_kmeans', 'd1_arterial_ph_min_kmeans', 'd1_creatinine_max_kmeans', 'wbc_apache_kmeans', 'd1_pao2fio2ratio_max_kmeans', 'h1_bun_min_kmeans', 'd1_sysbp_invasive

c:\program files\python37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


попытка № 0 успешная, счетчик обновится
len bad cols: 96
check_list_sign:  ['h1_hemaglobin_max_kmeans', 'h1_hco3_max_kmeans', 'h1_diasbp_noninvasive_max_kmeans', 'd1_mbp_invasive_max_kmeans', 'h1_bilirubin_max_kmeans', 'h1_creatinine_min_kmeans', 'h1_platelets_max_kmeans', 'h1_platelets_min_kmeans', 'd1_arterial_pco2_max_kmeans', 'h1_mbp_noninvasive_max_kmeans', 'h1_potassium_min_kmeans', 'd1_glucose_min_kmeans', 'h1_albumin_max_kmeans', 'd1_pao2fio2ratio_min_kmeans', 'h1_pao2fio2ratio_min_kmeans', 'ph_apache_kmeans', 'pao2_apache_kmeans', 'h1_inr_min_kmeans', 'h1_hematocrit_min_kmeans', 'h1_sodium_min_kmeans', 'bilirubin_apache_kmeans', 'h1_arterial_pco2_max_kmeans', 'd1_hco3_min_kmeans', 'd1_creatinine_min_kmeans', 'd1_albumin_max_kmeans', 'h1_sysbp_invasive_max_kmeans', 'h1_wbc_min_kmeans', 'd1_arterial_ph_min_kmeans', 'd1_creatinine_max_kmeans', 'wbc_apache_kmeans', 'd1_pao2fio2ratio_max_kmeans', 'h1_bun_min_kmeans', 'd1_sysbp_invasive_max_kmeans', 'd1_albumin_min_kmeans', 'd1_hema

...

c:\program files\python37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


попытка № 0 успешная, счетчик обновится
len bad cols: 81
check_list_sign:  ['h1_hemaglobin_max_kmeans', 'h1_hco3_max_kmeans', 'd1_mbp_invasive_max_kmeans', 'h1_bilirubin_max_kmeans', 'h1_creatinine_min_kmeans', 'h1_platelets_max_kmeans', 'h1_platelets_min_kmeans', 'd1_arterial_pco2_max_kmeans', 'h1_potassium_min_kmeans', 'd1_glucose_min_kmeans', 'h1_albumin_max_kmeans', 'd1_pao2fio2ratio_min_kmeans', 'h1_pao2fio2ratio_min_kmeans', 'ph_apache_kmeans', 'pao2_apache_kmeans', 'h1_inr_min_kmeans', 'h1_hematocrit_min_kmeans', 'bilirubin_apache_kmeans', 'h1_arterial_pco2_max_kmeans', 'd1_hco3_min_kmeans', 'd1_albumin_max_kmeans', 'h1_sysbp_invasive_max_kmeans', 'h1_wbc_min_kmeans', 'd1_arterial_ph_min_kmeans', 'd1_creatinine_max_kmeans', 'wbc_apache_kmeans', 'd1_pao2fio2ratio_max_kmeans', 'h1_bun_min_kmeans', 'd1_sysbp_invasive_max_kmeans', 'd1_albumin_min_kmeans', 'd1_hematocrit_min_kmeans', 'paco2_apache_kmeans', 'h1_diasbp_invasive_max_kmeans', 'creatinine_apache_kmeans', 'd1_diasbp_invasi

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# d = pd.read_csv("data/data_kmeans_replace.csv")
d.shape